## setting

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install datasets==2.14.0

In [ ]:
import os
import pandas as pd
import numpy as np
import random

In [ ]:
import torch
import time
from tqdm import tqdm
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
# Seed 설정
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 0
set_seed(seed)

## data

In [ ]:
path = '/kaggle/input/'
data = pd.read_json(path+'data.json')
sample_SEED = 0

In [ ]:
def split_data(data, sample_SEED, fixSEED=0):
  # normal bot split
  normal = data[data['restrict']==0].reset_index(drop=True)
  bot = data[data['restrict']==1].reset_index(drop=True)
  # normal underampling
  random.seed(sample_SEED)
  sample = random.sample(list(range(len(normal))), len(bot))
  normal = normal.loc[sample].reset_index(drop=True)
  data = pd.concat([normal, bot], axis=0)
  data['restrict'].value_counts()
  # train valid test split
  train, test = train_test_split(data, test_size=0.3, random_state=fixSEED, stratify = data['restrict'])
  train, valid = train_test_split(train, test_size=0.3, random_state=fixSEED, stratify = train['restrict'])
  train = train.reset_index(drop=True)
  valid = valid.reset_index(drop=True)
  test = test.reset_index(drop=True)
  # 1:1 fix
  random.seed(fixSEED)
  move = random.sample(list(valid[valid['restrict']==1].index), 1)
  test = pd.concat([test, valid.loc[move]], axis=0).reset_index(drop=True)
  valid = valid.drop(move, axis=0).reset_index(drop=True)
  return train, valid, test

# data split
train, valid, test = split_data(data, sample_SEED)

In [ ]:
import re

def extract_first_two_sentences(text):
    sentences = re.split(r'(?<=\.)\s+', text)
    return ' '.join(sentences[:2])

# Apply the function to the text_column and create a new column
train['base'] = train['longlongtext'].apply(extract_first_two_sentences)
valid['base'] = valid['longlongtext'].apply(extract_first_two_sentences)
test['base'] = test['longlongtext'].apply(extract_first_two_sentences)

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

train['tokenized'] = train['base'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
train['token_length'] = train['tokenized'].apply(len)

# Find the maximum token length
max_token_length = train['token_length'].max()

print(f"Maximum token length: {max_token_length}")

valid['tokenized'] = valid['base'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
valid['token_length'] = valid['tokenized'].apply(len)

# Find the maximum token length
max_token_length = valid['token_length'].max()

print(f"Maximum token length: {max_token_length}")

test['tokenized'] = test['base'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
test['token_length'] = test['tokenized'].apply(len)

# Find the maximum token length
max_token_length = test['token_length'].max()

print(f"Maximum token length: {max_token_length}")

## model setting

In [ ]:
# model, tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", max_length=max_token_length)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                              num_labels=2)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
from datasets import Dataset, DatasetDict

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['base'], padding='max_length', truncation=True, max_length=31)

# 전처리 함수
def prepare_data(df):
    data = {'base': df['base'].tolist(), 'restrict': df['restrict'].tolist()}
    dataset = Dataset.from_dict(data)
    dataset = dataset.map(tokenize_function, batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'restrict'])
    return dataset

# X, y 분리
X_train, y_train = train['base'], train['restrict']
X_val, y_val = valid['base'], valid['restrict']
X_test, y_test = test['base'], test['restrict']


train_df = pd.DataFrame(X_train)
train_df['restrict'] = y_train
val_df = pd.DataFrame(X_val)
val_df['restrict'] = y_val
test_df = pd.DataFrame(X_test)
test_df['restrict'] = y_test

train_dataset = prepare_data(train_df)
val_dataset = prepare_data(val_df)
test_dataset = prepare_data(test_df)

## modeling

In [ ]:
# batch size = 8
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 5

# Learning rate scheduler -> 학습률을 줄이거나 늘리면서 안정성+성능 개선 가능
num_training_steps = num_epochs * len(train_loader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

gradient_accumulation_steps = 8  # Accumulate gradients over 8 steps

# Training function
def train(model, train_loader, optimizer, scheduler):
    model.train()
    total_loss = 0
    scaler = torch.cuda.amp.GradScaler()

    for step, batch in enumerate(tqdm(train_loader, desc="Training")):
        with torch.cuda.amp.autocast():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['restrict'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss / gradient_accumulation_steps
            total_loss += loss.item()

        scaler.scale(loss).backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()

    return total_loss / len(train_loader)

# Evaluation function
def evaluate(model, data_loader):
    model.eval()
    preds, true_labels = [], []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['restrict'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds.extend(torch.argmax(logits, axis=1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return preds, true_labels

In [ ]:
# 훈련 시간 측정 시작
start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    train_loss = train(model, train_loader, optimizer, scheduler)
    epoch_end_time = time.time()

    print(f"Epoch {epoch + 1}, Loss: {train_loss}")
    print(f"Epoch {epoch + 1} Computing Time: {epoch_end_time - epoch_start_time:} seconds")

    # validation data 평가
    predicted_labels, true_labels = evaluate(model, val_loader)
    f1 = f1_score(true_labels, predicted_labels, zero_division=1)
    precision = precision_score(true_labels, predicted_labels, zero_division=1)
    recall = recall_score(true_labels, predicted_labels, zero_division=1)
    accuracy = accuracy_score(true_labels, predicted_labels)
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    print(f"Epoch {epoch + 1}, Validation F1 Score: {f1:}")
    print(f"Validation Precision: {precision:}")
    print(f"Validation Recall: {recall:}")
    print(f"Validation Accuracy: {accuracy:}")
    print("Validation Confusion Matrix:\n", conf_matrix)

# 훈련 시간 측정 종료
end_time = time.time()
total_training_time = end_time - start_time # 훈련시간

In [ ]:
# Final prediction
predicted_labels, true_labels = evaluate(model, test_loader)
f1 = f1_score(true_labels, predicted_labels, zero_division=1)
precision = precision_score(true_labels, predicted_labels, zero_division=1)
recall = recall_score(true_labels, predicted_labels, zero_division=1)
accuracy = accuracy_score(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Final Test Results:")
print(f"Test F1 Score: {f1:}")
print(f"Test Precision: {precision:}")
print(f"Test Recall: {recall:}")
print(f"Test Accuracy: {accuracy:}")
print("Test Confusion Matrix:\n", conf_matrix)
print(f"Total Computing Time: {total_training_time:} seconds")